In [ ]:
!pip install lime
!pip install transformers
!pip install pytreebank
!pip install datasets==1.5.0

import os
import random

import pandas as pd
import numpy as np
import scipy as sp
import torch
import spacy
from torch.utils.data import \
    TensorDataset, \
    DataLoader
from transformers import \
    BertTokenizer, \
    BertForSequenceClassification, \
    AdamW, \
    BertConfig, \
    get_linear_schedule_with_warmup


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd '/content/drive/MyDrive/NLP Project'

/content/drive/MyDrive/NLP Project


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# BERT Example

**Load your data**

In [6]:
import src.data.dataload
sst=src.data.dataload.load_sst()
train, val, test = sst.train_val_test

In [7]:
ag_news=src.data.dataload.load_agnews()
train_ag, _, test_ag = ag_news.train_val_test

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/17ec33e23df9e89565131f989e0fdf78b0cc4672337b582da83fc3c9f79fe34d. Subsequent calls will reuse this data.


**Set up your model & tokenizer**

In [8]:

bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Project/BERT Model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

id2label = bert_model.config.id2label
label2id = bert_model.config.label2id
labels = sorted(label2id, key=label2id.get)

**Explanations on SST**

In [29]:
import importlib #this was needed only because I was modifying the modules
import lime_explainer #this was needed only because I was modifying the modules
importlib.reload(lime_explainer) #this was needed only because I was modifying the modules

<module 'lime_explainer' from '/content/drive/MyDrive/NLP Project/lime_explainer.py'>

In [ ]:
from lime_explainer import *

from lime.lime_text import LimeTextExplainer

LIME_explainer = Explainer(bert_model,tokenizer,labels,device,'LIME','BERT')

indices=np.random.choice(len(test), 20, replace=False)
instance_array = test['sentence'].iloc[indices]

top_tokens_SST,top_values_SST=LIME_explainer.explain_instances(instance_array)

In [32]:
top_tokens_SST,top_values_SST

([['if', 'of', 'sweet', 'widely', 'vital', 'San', 'one', 'most', 'least', 's'],
  ['smoothly',
   'entire',
   'slight',
   'should',
   'off',
   'go',
   'punches',
   'indeed',
   'the',
   's'],
  ['Sometimes', 's', 'enough', 'that'],
  ['lacks', 'punch', 'story', 'The', 'central'],
  ['blend',
   'preciousness',
   'usual',
   'on',
   'cleverness',
   'shares',
   'facile',
   'Based',
   'the',
   'Leavitt'],
  ['contrived',
   'sole',
   'where',
   'performances',
   'actors',
   'in',
   'only',
   'those',
   'like',
   'the'],
  ['pound', 'away', 'just'],
  ['than',
   'implausible',
   'blithely',
   'French',
   'sitcom',
   'with',
   'a',
   'loose',
   'strays',
   'Director'],
  ['only',
   'Fatale',
   'be',
   'element',
   'the',
   'surprise',
   'for',
   'thing',
   'might',
   'has'],
  ['camouflaged',
   'activities',
   'as',
   'everyday',
   'precious',
   'artfully',
   'increments',
   'A',
   'film',
   'of'],
  ['gorgeous',
   'splendidly',
   'mummifie

**Explanations on AG News**

In [ ]:
'''
NOTE - the BERT model hasn't been fine-tuned on AG News so this is just to show the mechanics on a different dataset
'''
LIME_explainer = Explainer(bert_model,tokenizer,labels,device,'LIME','BERT')
indices=np.random.choice(len(test_ag), 20, replace=False)
instance_array = train_ag['sentence'].iloc[indices]
top_tokens_AG,top_values_AG=LIME_explainer.explain_instances(instance_array)

In [34]:
top_tokens_AG,top_values_AG

([['begin',
   'million',
   'nearly',
   'Google',
   'complete',
   'for',
   'approval',
   'afternoon',
   'of',
   'this'],
  ['s',
   'between',
   'their',
   'negotiations',
   'for',
   'ite',
   'handover',
   'that',
   'and',
   'Najaf'],
  ['lifelong',
   'Bush',
   'with',
   'of',
   'Democrat',
   '2004',
   'sanity',
   'said',
   'stance',
   '151'],
  ['focus',
   'Wall',
   'steady',
   'Materials',
   'Tuesday',
   'and',
   'Street',
   'prices',
   'Applied',
   'Reuters'],
  ['till',
   'owners',
   'while',
   'Windows',
   'be',
   'll',
   'patch',
   'started',
   'XP',
   'a'],
  ['honorable',
   'great',
   'around',
   'tarnished',
   'Paul',
   'ATHENS',
   'that',
   'quot',
   'right',
   'have'],
  ['ensured',
   'Mizuki',
   'pulled',
   'marathon',
   'holder',
   'Noguchi',
   'as',
   'record',
   'up',
   'end'],
  ['lost',
   'Chanda',
   'ended',
   'and',
   'bid',
   'on',
   'Sun',
   '1',
   'Williams',
   'Li'],
  ['out',
   'withdrew',
  

# BCN Example

**Set up your model & tokenizer**

In [35]:
cd /content/drive/MyDrive/NLP Project/AllenNLP

/content/drive/MyDrive/NLP Project/AllenNLP


In [ ]:
!pip install allennlp==2.1.0 allennlp-models==2.1.0
import spacy
nlp = spacy.load('en_core_web_sm')
import allennlp
import allennlp_models
from allennlp.models.archival import load_archive
from allennlp.common.util import JsonDict
from allennlp.data import Instance
from allennlp.predictors.predictor import Predictor
from allennlp.data.fields import LabelField
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
from typing import List, Dict
from overrides import overrides

#BCN_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Project/BERT Model')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#id2label = bert_model.config.id2label
#label2id = bert_model.config.label2id
#labels = sorted(label2id, key=label2id.get)

In [39]:
# importing the dataset reader
import tagging
# importing the BCN model
import BCN_model
archive = load_archive("./BCN_output/model.tar.gz")
BCN_model = archive.model
vocab = BCN_model.vocab
BCN_predictor = Predictor.from_archive(archive, 'ag_text_classifier')

**Explanations on SST**

In [44]:
cd '/content/drive/MyDrive/NLP Project'

/content/drive/MyDrive/NLP Project


In [47]:
import importlib #this was needed only because I was modifying the modules
import lime_explainer #this was needed only because I was modifying the modules
importlib.reload(lime_explainer) #this was needed only because I was modifying the modules

from lime_explainer import *

labels_BCN = ['Sci/Tech', 'Sports','World','Business']

LIME_explainer = Explainer(BCN_predictor,None,labels_BCN,device,'LIME','BCN')

indices=np.random.choice(len(test), 20, replace=False)
instance_array = test['sentence'].iloc[indices]

top_tokens_SST,top_values_SST=LIME_explainer.explain_instances(instance_array)


#indices=np.random.choice(len(test_ag), 20, replace=False)
#instance_array = test_ag['sentence'].iloc[indices]
#top_tokens_AG,top_values_AG=explain_instances(instance_array,exp,predict_probas)

['Topkapi', 'this', 'not', 'is'] [0.3753649453163358, 0.03616457622495696, -0.03395436904051829, -0.005531189261271591]
['team', 'montied', 'jovial', 'scrappy', 'that', 'thing', 'could', 'interesting', 'are', 'they'] [0.2546293647960733, -0.18585059474570678, 0.1123566643177693, 0.0763229340389504, 0.056783464143029865, 0.05491639094347341, 0.04992165438709392, -0.043099075743672936, -0.04061711462024325, -0.03783445172838989]
['tells', 'help', 'that', 'Jeong', 'I', 't', 'it', 'captivated', 'and', 'beautifully'] [-0.11348402266718109, 0.1014395039660069, -0.09145565485264642, -0.07484864396129721, 0.07091443920875591, 0.06862883117448626, 0.054211301989145214, -0.05369019536281044, 0.04409449982139864, 0.024950824337315512]
['regimes', 'abuses', 'It', 's', 'Latin', 'ultimately', 'an', 'stands', 'important', 'oppressive'] [0.06107674380007077, 0.05811947684748645, -0.05139489555599274, 0.04647308284012993, 0.04542765642186966, -0.043768170815008875, -0.03318385282411329, 0.0317118557441

In [48]:
top_tokens_SST,top_values_SST

([['Topkapi', 'this', 'not', 'is'],
  ['team',
   'montied',
   'jovial',
   'scrappy',
   'that',
   'thing',
   'could',
   'interesting',
   'are',
   'they'],
  ['tells',
   'help',
   'that',
   'Jeong',
   'I',
   't',
   'it',
   'captivated',
   'and',
   'beautifully'],
  ['regimes',
   'abuses',
   'It',
   's',
   'Latin',
   'ultimately',
   'an',
   'stands',
   'important',
   'oppressive'],
  ['ups',
   'artists',
   'waltzes',
   'pretense',
   'heavy',
   'creating',
   'between',
   'off',
   'and',
   'the'],
  ['deeply',
   'thriller',
   'comic',
   'eccentric',
   'little',
   'personality',
   'love',
   'An',
   'own',
   'in'],
  ['pay',
   're',
   'stories',
   'but',
   'attention',
   'each',
   'to',
   'all',
   'they',
   'have'],
  ['gourmet',
   'coming',
   'drive',
   'fare',
   'not',
   'from',
   'is',
   'It',
   'fair',
   'the'],
  ['Gantzes',
   'tend',
   'guys',
   'let',
   'hook',
   'off',
   'interviews',
   'to',
   'the',
   'The'],
  

**Explanations on AG News**

In [50]:
LIME_explainer = Explainer(BCN_predictor,None,labels_BCN,device,'LIME','BCN')
indices=np.random.choice(len(test_ag), 20, replace=False)
instance_array = test_ag['sentence'].iloc[indices]
top_tokens_AG,top_values_AG=LIME_explainer.explain_instances(instance_array)

['mobiles', 'There', 'Nokia', 'handsets', 'conducted', 'or', 'Finnish', 'for', 'phones', 'using'] [0.03319802028492146, -0.029439526671869085, 0.024304741854920063, 0.023132410351267934, 0.021788521824297687, -0.02016218256571348, 0.01685358803547302, -0.016423099234165833, -0.013768791635199736, 0.00657328045006203]
['said', 'number', 'minister', 'in', 'Poland', 'Iraq', 'significantly', 'its', 'end', 'of'] [0.06826413480971452, -0.06564397083230183, 0.05197873587692893, 0.05165912490838034, 0.05021307790321247, 0.050102533283465034, 0.046269308181636826, 0.043516987083678155, -0.04328885167063657, -0.03693546479754064]
['AP', 'was', 'lineup', 'Troy', '60', 'Anaheim', 'back', 'Twins', 'from', 'the'] [0.0828098349933068, 0.05866351912380532, 0.05782373415548009, 0.048598279641914745, -0.03178479139873425, 0.03094813967557778, -0.02086023715268655, -0.01358566150582394, 0.007214192442564138, -0.0025604169318306936]
['Acrobat', 'Yahoo', 'Adobe', 'At', 'million', 'halo', 'glow', 'effect', 

**Helper functions (model dependent)**

In [38]:
@Predictor.register('ag_text_classifier')
class AGNewsClassifier(Predictor):
    """
    Predictor for any model that takes in a sentence and returns
    a single class for it.  In particular, it can be used with
    the [`BasicClassifier`](../models/basic_classifier.md) model.

    """

    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"Description": sentence})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        """
        Expects JSON that looks like `{"sentence": "..."}`.
        Runs the underlying model, and adds the `"label"` to the output.
        """
        sentence = json_dict["Description"]
        reader_has_tokenizer = (
            getattr(self._dataset_reader, "tokenizer", None) is not None
            or getattr(self._dataset_reader, "_tokenizer", None) is not None
        )
        if not reader_has_tokenizer:
            tokenizer = SpacyTokenizer()
            sentence = tokenizer.tokenize(sentence)
        return self._dataset_reader.text_to_instance(sentence)

    @overrides
    def predictions_to_labeled_instances(
        self, instance: Instance, outputs: Dict[str, np.ndarray]
    ) -> List[Instance]:
        new_instance = instance.duplicate()
        label = np.argmax(outputs["class_probabilities"])
        new_instance.add_field("label", LabelField(int(label), skip_indexing=True))
        return [new_instance]